In [18]:
import yfinance as yf

def get_ticker_info(symbol):
    ticker = yf.Ticker(symbol)
    ticker.history(start="2020-01-01", end="2020-12-31",period="1d")
    info = ticker.info
    return info

# Usage
info = get_ticker_info("AAPL")
print(info)

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add

/Users/hanyuwu/Library/Caches/pypoetry/virtualenvs/compare-multiple-backtests-f1m3Q6xX-py3.11/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [ ]:
for i in info.keys():
    print(i, info[i])

In [26]:
# Simplified daily price changes (%) for a stock over 10 days
import random

price_changes = [round(random.uniform(-2.0, 2.0), 2) for _ in range(100)]


In [28]:
import random

def initialize_population(size, gene_length):
    return [[random.choice([-1, 0, 1]) for _ in range(gene_length)] for _ in range(size)]


In [29]:
import numpy as np

def calculate_fitness(individual, price_changes):
    # Calculate daily returns based on trading actions
    daily_returns = [action * change for action, change in zip(individual, price_changes)]
    
    # Calculate the average daily return and standard deviation of daily returns
    avg_daily_return = np.mean(daily_returns)
    std_dev_returns = np.std(daily_returns)
    
    # Assuming no risk-free rate for simplicity; subtract risk-free rate from avg_daily_return if available
    if std_dev_returns > 0:
        sharpe_ratio = avg_daily_return / std_dev_returns
    else:
        sharpe_ratio = 0  # To handle division by zero if std_dev_returns is 0
    
    return sharpe_ratio


In [30]:
def select_parents(population, fitnesses, num_parents):
    parents = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)
    return [parent for parent, _ in parents[:num_parents]]


In [31]:
def crossover(parents, offspring_size):
    offspring = []
    crossover_point = random.randint(1, offspring_size - 1)
    for _ in range(offspring_size):
        parent1, parent2 = random.sample(parents, 2)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        offspring.append(child)
    return offspring


In [32]:
def mutate(offspring, mutation_rate=0.1):
    for individual in offspring:
        if random.random() < mutation_rate:
            mutation_point = random.randint(0, len(individual) - 1)
            individual[mutation_point] = random.choice([-1, 0, 1])
    return offspring


In [36]:
def run_genetic_algorithm(price_changes, generations=50, population_size=100000, gene_length=100, num_parents=300):
    population = initialize_population(population_size, gene_length)
    for generation in range(generations):
        fitnesses = [calculate_fitness(individual, price_changes) for individual in population]
        parents = select_parents(population, fitnesses, num_parents)
        offspring = crossover(parents, offspring_size=len(population) - len(parents))
        offspring = mutate(offspring)
        population = parents + offspring
        print(f"Generation {generation + 1}, Best Fitness: {max(fitnesses)}")

    best_index = fitnesses.index(max(fitnesses))
    return population[best_index]

# Running the GA with our sample data
best_strategy = run_genetic_algorithm(price_changes)
print("Best Trading Strategy:", best_strategy)


Generation 1, Best Fitness: 0.41699046793700284
Generation 2, Best Fitness: 0.4584295811700599
Generation 3, Best Fitness: 0.5010539820617976
Generation 4, Best Fitness: 0.5328474900677242
Generation 5, Best Fitness: 0.5835367236690455
Generation 6, Best Fitness: 0.6197849262762906
Generation 7, Best Fitness: 0.6798239754948533
Generation 8, Best Fitness: 0.7033017788168593
Generation 9, Best Fitness: 0.7734731137075797
Generation 10, Best Fitness: 0.818077444437067
Generation 11, Best Fitness: 0.8375504742714992
Generation 12, Best Fitness: 0.8685554242920289
Generation 13, Best Fitness: 0.9177304952507217
Generation 14, Best Fitness: 0.9594939124598625
Generation 15, Best Fitness: 0.9983874970929032
Generation 16, Best Fitness: 1.01764897427458
Generation 17, Best Fitness: 1.0524507181555047
Generation 18, Best Fitness: 1.0737046714940113
Generation 19, Best Fitness: 1.1037729767317326
Generation 20, Best Fitness: 1.127988026928608
Generation 21, Best Fitness: 1.1524373683093565
Gene

In [ ]:


# Calculate daily returns based on actions
# Here, buying (1) means we gain if the price increases, selling (-1) means we gain if the price decreases.
daily_returns = [action * change/100 for action, change in zip(actions, price_changes)]


In [ ]:
def cumsum(lst):
    total = 0
    cumsum_list = []
    for num in lst:
        total += num
        cumsum_list.append(total)
    return cumsum_list

In [ ]:
cumsum_returns = cumsum(daily_returns)

In [ ]:
cumsum_returns